# Movie Review Prediction System

This movie review prediction system is implemented using the Graph4NLP library available at https://github.com/graph4ai/graph4nlp

The movie review dataset https://www.cs.cornell.edu/people/pabo/movie-review-data/ has been pre-processed and split into train, valid and test set. It is stored in the folder `/data/MRD/raw`

In order to run the training and testing code, please follow the instructions below to install necessary packages from Graph4NLP library.

## Graph4NLP library environment setup


### Install the required dependencies:
```
torch==1.6.0
torchtext # >=0.7.0
dgl==0.4.3.post2
networkx==2.4
nltk==3.4.5
numpy==1.17.4
PyYAML==5.3
scikit-learn==0.23.1
scipy==1.4.1
stanfordcorenlp==3.9.1.1
tqdm==4.47.0
pythonds==1.2.1
```

### Create virtual environment
```
conda create --name graph4nlp python=3.8
conda activate graph4nlp
```

### Install graph4nlp library


#### Step 1: Clone the github repo of `Graph4NLP`:
```bash
git clone -b stable_nov2021 https://github.com/graph4ai/graph4nlp.git
cd graph4nlp
```

#### Step 2: Do configuration

Then run `./configure` (or `./configure.bat`  if you are using Windows 10) to config your installation. The configuration program will ask you to specify your CUDA version. If you do not have a GPU, please type 'cpu'.
```bash
./configure
```

#### Step 3: Finally, install the package:

Finally, install the package:

```shell
python setup.py install
```

```

## Set up StanfordCoreNLP (for static graph construction)

#### Step 1: Download StanfordCoreNLP
https://stanfordnlp.github.io/CoreNLP/

#### Step 2: Go to the root folder and start the server
```
java -mx4g -cp "*" edu.stanford.nlp.pipeline.StanfordCoreNLPServer -port 9000 -timeout 15000

In [ ]:
import argparse
import os
import time
import numpy as np
import torch
import torch.backends.cudnn as cudnn
import torch.multiprocessing
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import yaml
from torch.optim.lr_scheduler import ReduceLROnPlateau
from torch.utils.data import DataLoader
from sklearn.metrics import r2_score

from graph4nlp.datasets.MRD import MRDDataset
from graph4nlp.modules.graph_construction import (
    ConstituencyBasedGraphConstruction,
    DependencyBasedGraphConstruction,
    IEBasedGraphConstruction,
    NodeEmbeddingBasedGraphConstruction,
    NodeEmbeddingBasedRefinedGraphConstruction,
)
from graph4nlp.modules.graph_construction.embedding_construction import WordEmbedding
from graph4nlp.modules.graph_embedding import GAT, GGNN, GraphSAGE
# from feedforward_nn import NNclassifier
from graph4nlp.modules.utils import constants as Constants
from graph4nlp.modules.utils.generic_utils import EarlyStopping, grid, to_cuda
from graph4nlp.modules.utils.logger import Logger

In [2]:
class TextClassifier(nn.Module):
    def __init__(self, vocab, label_model, cfg):
        super(TextClassifier, self).__init__()
        self.cfg = cfg
        self.vocab = vocab
        self.label_model = label_model
        embedding_style = {
            "single_token_item": True if cfg["graph_type"] != "ie" else False,
            "emb_strategy": cfg.get("emb_strategy", "w2v_bilstm"),
            "num_rnn_layers": 1,
            "bert_model_name": cfg.get("bert_model_name", "bert-base-uncased"),
            "bert_lower_case": True,
        }

        assert not (
            cfg["graph_type"] in ("node_emb", "node_emb_refined") and cfg["gnn"] == "gat"
        ), "dynamic graph construction does not support GAT"

        use_edge_weight = False
        if cfg["graph_type"] == "dependency":
            self.graph_topology = DependencyBasedGraphConstruction(
                embedding_style=embedding_style,
                vocab=vocab.in_word_vocab,
                hidden_size=cfg["num_hidden"],
                word_dropout=cfg["word_dropout"],
                rnn_dropout=cfg["rnn_dropout"],
                fix_word_emb=not cfg["no_fix_word_emb"],
                fix_bert_emb=not cfg.get("no_fix_bert_emb", False),
            )
        elif cfg["graph_type"] == "constituency":
            self.graph_topology = ConstituencyBasedGraphConstruction(
                embedding_style=embedding_style,
                vocab=vocab.in_word_vocab,
                hidden_size=cfg["num_hidden"],
                word_dropout=cfg["word_dropout"],
                rnn_dropout=cfg["rnn_dropout"],
                fix_word_emb=not cfg["no_fix_word_emb"],
                fix_bert_emb=not cfg.get("no_fix_bert_emb", False),
            )
        elif cfg["graph_type"] == "ie":
            self.graph_topology = IEBasedGraphConstruction(
                embedding_style=embedding_style,
                vocab=vocab.in_word_vocab,
                hidden_size=cfg["num_hidden"],
                word_dropout=cfg["word_dropout"],
                rnn_dropout=cfg["rnn_dropout"],
                fix_word_emb=not cfg["no_fix_word_emb"],
                fix_bert_emb=not cfg.get("no_fix_bert_emb", False),
            )
            raise RuntimeError("Unknown graph_type: {}".format(cfg["graph_type"]))

        if "w2v" in self.graph_topology.embedding_layer.word_emb_layers:
            self.word_emb = self.graph_topology.embedding_layer.word_emb_layers[
                "w2v"
            ].word_emb_layer
        else:
            self.word_emb = WordEmbedding(
                self.vocab.in_word_vocab.embeddings.shape[0],
                self.vocab.in_word_vocab.embeddings.shape[1],
                pretrained_word_emb=self.vocab.in_word_vocab.embeddings,
                fix_emb=not cfg["no_fix_word_emb"],
            ).word_emb_layer

        if cfg["gnn"] == "gat":
            heads = [cfg["gat_num_heads"]] * (cfg["gnn_num_layers"] - 1) + [
                cfg["gat_num_out_heads"]
            ]
            self.gnn = GAT(
                cfg["gnn_num_layers"],
                cfg["num_hidden"],
                cfg["num_hidden"],
                cfg["num_hidden"],
                heads,
                direction_option=cfg["gnn_direction_option"],
                feat_drop=cfg["gnn_dropout"],
                attn_drop=cfg["gat_attn_dropout"],
                negative_slope=cfg["gat_negative_slope"],
                residual=cfg["gat_residual"],
                activation=F.elu,
                allow_zero_in_degree=True,
            )
        elif cfg["gnn"] == "graphsage":
            self.gnn = GraphSAGE(
                cfg["gnn_num_layers"],
                cfg["num_hidden"],
                cfg["num_hidden"],
                cfg["num_hidden"],
                cfg["graphsage_aggreagte_type"],
                direction_option=cfg["gnn_direction_option"],
                feat_drop=cfg["gnn_dropout"],
                bias=True,
                norm=None,
                activation=F.relu,
                use_edge_weight=use_edge_weight,
            )
        elif cfg["gnn"] == "ggnn":
            self.gnn = GGNN(
                cfg["gnn_num_layers"],
                cfg["num_hidden"],
                cfg["num_hidden"],
                cfg["num_hidden"],
                feat_drop=cfg["gnn_dropout"],
                direction_option=cfg["gnn_direction_option"],
                bias=True,
                use_edge_weight=use_edge_weight,
            )
        else:
            raise RuntimeError("Unknown gnn type: {}".format(cfg["gnn"]))

        self.clf = NNclassifier(
            2 * cfg["num_hidden"]
            if cfg["gnn_direction_option"] == "bi_sep"
            else cfg["num_hidden"],
            cfg["num_classes"],
            [cfg["num_hidden"]],
            graph_pool_type=cfg["graph_pooling"],
            dim=cfg["num_hidden"],
            use_linear_proj=cfg["max_pool_linear_proj"],
        )

        self.loss = nn.MSELoss(size_average=None, reduce=None, reduction="mean")

    def forward(self, graph_list, tgt=None, require_loss=True):
        # graph embedding construction
        batch_gd = self.graph_topology(graph_list)

        # run GNN
        self.gnn(batch_gd)

        # run graph classifier
        self.clf(batch_gd)
        logits = batch_gd.graph_attributes["logits"]

        if require_loss:
            logits = logits.to(torch.float32)
            loss = self.loss(logits.view(-1), tgt)
            return logits, loss
        else:
            return logits

    @classmethod
    def load_checkpoint(cls, model_path):
        """The API to load the model.

        Parameters
        ----------
        model_path : str
            The saved model path.

        Returns
        -------
        Class
        """
        return torch.load(model_path)

In [3]:
import collections
from torch import nn

from graph4nlp.modules.utils.base import GraphClassifierBase, GraphClassifierLayerBase
from graph4nlp.modules.utils.avg_pooling import AvgPooling
from graph4nlp.modules.utils.max_pooling import MaxPooling

class NNclassifier(GraphClassifierBase):
    r"""NNclassifier class for graph classification task.

    Parameters
    ----------
    input_size : int
        The dimension of input graph embeddings.
    num_class : int
        The number of classes for classification.
    hidden_size : list of int
        Hidden size per NN layer.
    activation: nn.Module, optional
        The activation function, default: `nn.ReLU()`.
    """

    def __init__(
        self,
        input_size,
        num_class,
        hidden_size,
        activation=None,
        graph_pool_type="max_pool",
        **kwargs
    ):
        super(NNclassifier, self).__init__()

        if not activation:
            activation = nn.ReLU()

        if graph_pool_type == "avg_pool":
            self.graph_pool = AvgPooling()
        elif graph_pool_type == "max_pool":
            self.graph_pool = MaxPooling(**kwargs)
        else:
            raise RuntimeError("Unknown graph pooling type: {}".format(graph_pool_type))

        self.classifier = NNclassifierLayer(input_size, num_class, hidden_size, activation)

    def forward(self, graph):
        r"""Compute the logits tensor for graph classification.

        Parameters
        ----------
        graph : GraphData
            The graph data containing graph embeddings.

        Returns
        -------
        list of GraphData
            The output graph data containing logits tensor for graph classification.
        """
        graph_emb = self.graph_pool(graph, "node_emb")
        logits = self.classifier(graph_emb)
        graph.graph_attributes["logits"] = logits

        return graph
    
    
class NNclassifierLayer(GraphClassifierLayerBase):
    r"""NNclassifierLayer class for graph classification task.

    Parameters
    ----------
    input_size : int
        The dimension of input graph embeddings.
    num_class : int
        The number of classes for classification.
    hidden_size : list of int
        Hidden size per NN layer.
    activation: nn.Module, optional
        The activation function, default: `nn.ReLU()`.
    """

    def __init__(self, input_size, num_class, hidden_size, activation=None):
        super(NNclassifierLayer, self).__init__()

        if not activation:
            activation = nn.ReLU()

        # build the linear module list
        module_seq = []

        for layer_idx in range(len(hidden_size)):
            if layer_idx == 0:
                module_seq.append(
                    ("fc" + str(layer_idx), nn.Linear(input_size, hidden_size[layer_idx]))
                )
            else:
                module_seq.append(
                    (
                        "fc" + str(layer_idx),
                        nn.Linear(hidden_size[layer_idx - 1], self.hidden_size[layer_idx]),
                    )
                )

            module_seq.append(("activate" + str(layer_idx), activation))

        module_seq.append(("fc_end", nn.Linear(hidden_size[-1], num_class)))

        self.classifier = nn.Sequential(collections.OrderedDict(module_seq))

    def forward(self, graph_emb):
        r"""Compute the logits tensor for graph classification.

        Parameters
        ----------
        graph_emb : torch.Tensor
            The input graph embeddings.

        Returns
        -------
        torch.Tensor
            The output logits tensor for graph classification.
        """
        return self.classifier(graph_emb)

In [18]:
def trainer(model, cfg, logger, train_dataloader, val_dataloader, optimizer, scheduler, stopper):
    dur = []
    for epoch in range(cfg["epochs"]):
        model.train()
        train_loss = []
        train_acc = []
        t0 = time.time()
        for data in train_dataloader:
            tgt = to_cuda(data["tgt_tensor"], cfg["device"])
            data["graph_data"] = data["graph_data"].to(cfg["device"])
            logits, loss = model(data["graph_data"], tgt, require_loss=True)

            # add graph regularization loss if available
            if data["graph_data"].graph_attributes.get("graph_reg", None) is not None:
                loss = loss + data["graph_data"].graph_attributes["graph_reg"]

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            train_loss.append(loss.item())

            pred = logits.cpu()
            train_acc.append(r2_score(tgt.cpu(), pred.detach().cpu().view(-1)))
            dur.append(time.time() - t0)

        val_acc = evaluate(model, val_dataloader)
        scheduler.step(val_acc)
        print(
            "Epoch: [{} / {}] | Time: {:.2f}s | Loss: {:.4f} |"
            "Train Acc: {:.4f} | Val Acc: {:.4f}".format(
                epoch + 1,
                cfg["epochs"],
                np.mean(dur),
                np.mean(train_loss),
                np.mean(train_acc),
                val_acc,
            )
        )
        logger.write(
            "Epoch: [{} / {}] | Time: {:.2f}s | Loss: {:.4f} |"
            "Train Acc: {:.4f} | Val Acc: {:.4f}".format(
                epoch + 1,
                cfg["epochs"],
                np.mean(dur),
                np.mean(train_loss),
                np.mean(train_acc),
                val_acc,
            )
        )

        if stopper.step(val_acc, model):
            break

    return stopper.best_score

def evaluate(model, dataloader):
    model.eval()
    with torch.no_grad():
        pred_collect = []
        gt_collect = []
        for data in dataloader:
            tgt = to_cuda(data["tgt_tensor"], cfg["device"])
            data["graph_data"] = data["graph_data"].to(cfg["device"])
            logits = model(data["graph_data"], require_loss=False)
            pred_collect.append(logits)
            gt_collect.append(tgt)

        pred_collect = torch.cat(pred_collect, 0).cpu()
        gt_collect = torch.cat(gt_collect, 0).cpu()
        score = r2_score(gt_collect, pred_collect)

        return score

def tester(model, cfg, logger, test_dataloader, num_test, stopper):
    # restored best saved model
    model = TextClassifier.load_checkpoint(stopper.save_model_path)

    t0 = time.time()
    acc = evaluate(model, test_dataloader)
    dur = time.time() - t0
    print(
        "Test examples: {} | Time: {:.2f}s |  Test Acc: {:.4f}".format(num_test, dur, acc)
    )
    logger.write(
        "Test examples: {} | Time: {:.2f}s |  Test Acc: {:.4f}".format(num_test, dur, acc)
    )

    return acc

In [19]:
def main(cfg):
    # configure
    np.random.seed(cfg["seed"])
    torch.manual_seed(cfg["seed"])

    cfg["device"] = torch.device("cuda" if cfg["gpu"] < 0 else "cuda:%d" % -1)
    torch.cuda.manual_seed(cfg["seed"])
    torch.cuda.manual_seed_all(cfg["seed"])
    torch.backends.cudnn.deterministic = True
    cudnn.benchmark = False

    print("\n" + cfg["out_dir"])
    
    '''
    build logger
    '''
    logger = Logger(
        cfg["out_dir"],
        config={k: v for k, v in cfg.items() if k != "device"},
        overwrite=True)
    logger.write(cfg["out_dir"])
    
    '''
    build dataloader
    '''
    dynamic_init_topology_builder = None
    if cfg["graph_type"] == "dependency":
        topology_builder = DependencyBasedGraphConstruction
        graph_type = "static"
        merge_strategy = "tailhead"
    elif cfg["graph_type"] == "constituency":
        topology_builder = ConstituencyBasedGraphConstruction
        graph_type = "static"
        merge_strategy = "tailhead"
    elif cfg["graph_type"] == "ie":
        topology_builder = IEBasedGraphConstruction
        graph_type = "static"
        merge_strategy = "global"

        if cfg["init_graph_type"] == "dependency":
            dynamic_init_topology_builder = DependencyBasedGraphConstruction
        elif cfg["init_graph_type"] == "constituency":
            dynamic_init_topology_builder = ConstituencyBasedGraphConstruction
        elif cfg["init_graph_type"] == "ie":
            merge_strategy = "global"
            dynamic_init_topology_builder = IEBasedGraphConstruction
        else:
            # dynamic_init_topology_builder
            raise RuntimeError("Define your own dynamic_init_topology_builder")
    else:
        raise RuntimeError("Unknown graph_type: {}".format(cfg["graph_type"]))

    topology_subdir = "{}_graph".format(cfg["graph_type"])
    if cfg["graph_type"] == "node_emb_refined":
        topology_subdir += "_{}".format(cfg["init_graph_type"])

    dataset = MRDDataset(
        root_dir=cfg.get("root_dir", "data/MRD"),
        pretrained_word_emb_name=cfg.get("pretrained_word_emb_name", "840B"),
        pretrained_word_emb_cache_dir=cfg.get(
            "pretrained_word_emb_cache_dir", ".vector_cache"
        ),
        merge_strategy=merge_strategy,
        seed=cfg["seed"],
        thread_number=4,
        port=9000,
        timeout=15000,
        word_emb_size=300,
        graph_type=graph_type,
        topology_builder=topology_builder,
        topology_subdir=topology_subdir,
        dynamic_graph_type=cfg["graph_type"]
        if cfg["graph_type"] in ("node_emb", "node_emb_refined")
        else None,
        dynamic_init_topology_builder=dynamic_init_topology_builder,
        dynamic_init_topology_aux_args={"dummy_param": 0},
        for_inference=False,
        reused_vocab_model=None,
        reused_label_model=None,
    )


    train_dataloader = DataLoader(
        dataset.train,
        batch_size=cfg["batch_size"],
        shuffle=True,
        num_workers=cfg["num_workers"],
        collate_fn=dataset.collate_fn,
    )
    if not hasattr(dataset, "val"):
        dataset.val = dataset.test
    val_dataloader = DataLoader(
        dataset.val,
        batch_size=cfg["batch_size"],
        shuffle=False,
        num_workers=cfg["num_workers"],
        collate_fn=dataset.collate_fn,
    )
    test_dataloader = DataLoader(
        dataset.test,
        batch_size=cfg["batch_size"],
        shuffle=False,
        num_workers=cfg["num_workers"],
        collate_fn=dataset.collate_fn,
    )
    
    vocab = dataset.vocab_model
    label_model = dataset.label_model
    cfg["num_classes"] = 1 # label_model.num_classes
    num_train = len(dataset.train)
    num_val = len(dataset.val)
    num_test = len(dataset.test)
    print(
        "Train size: {}, Val size: {}, Test size: {}".format(
            num_train, num_val, num_test
        )
    )
    logger.write(
        "Train size: {}, Val size: {}, Test size: {}".format(
            num_train, num_val, num_test
        )
    )
    
    
    '''
    build model
    '''
    model = TextClassifier(vocab, label_model, cfg).to(cfg["device"])
    
    
    '''
    build optimizer
    '''
    parameters = [p for p in model.parameters() if p.requires_grad]
    optimizer = optim.Adam(parameters, lr=cfg["lr"])
    stopper = EarlyStopping(
        os.path.join(
            cfg["out_dir"],
            cfg.get("model_ckpt_name", Constants._SAVED_WEIGHTS_FILE)
        ),
        patience=cfg["patience"],
    )
    scheduler = ReduceLROnPlateau(
        optimizer,
        mode="max",
        factor=cfg["lr_reduce_factor"],
        patience=cfg["lr_patience"],
        verbose=True,
    )

    '''
    start training and testing
    '''
    t0 = time.time()
    
    val_acc = trainer(model, cfg, logger, train_dataloader, val_dataloader, optimizer, scheduler, stopper)
    test_acc = tester(model, cfg, logger, test_dataloader, num_test, stopper)

    runtime = time.time() - t0
    print("Total runtime: {:.2f}s".format(runtime))
    logger.write("Total runtime: {:.2f}s\n".format(runtime))
    logger.close()

    return val_acc, test_acc

In [20]:
import platform

parser = argparse.ArgumentParser()
parser.add_argument(
    "-config", "--config", default="config_doc/MRD/ggnn_bi_fuse_constituency.yaml", type=str, help="path to the config file"
)
args = vars(parser.parse_args([]))
with open(args["config"], "r") as setting:
    cfg = yaml.safe_load(setting)
    main(cfg)



out/MRD/ggnn_bi_fuse_dependency_ckpt
Loading pre-built label mappings stored in data/MRD/processed/dependency_graph/label.pt
Train size: 2975, Val size: 990, Test size: 990
[ Fix word embeddings ]
Epoch: [1 / 500] | Time: 8.72s | Loss: 0.0513 |Train Acc: -0.5676 | Val Acc: 0.0743
Saved model to out/MRD/ggnn_bi_fuse_dependency_ckpt/params.saved
Epoch: [2 / 500] | Time: 8.56s | Loss: 0.0262 |Train Acc: 0.2130 | Val Acc: 0.2233
Saved model to out/MRD/ggnn_bi_fuse_dependency_ckpt/params.saved
Epoch: [3 / 500] | Time: 8.53s | Loss: 0.0232 |Train Acc: 0.3065 | Val Acc: 0.4040
Saved model to out/MRD/ggnn_bi_fuse_dependency_ckpt/params.saved
Epoch: [4 / 500] | Time: 8.51s | Loss: 0.0200 |Train Acc: 0.3956 | Val Acc: 0.4742
Saved model to out/MRD/ggnn_bi_fuse_dependency_ckpt/params.saved
Epoch: [5 / 500] | Time: 8.56s | Loss: 0.0201 |Train Acc: 0.3961 | Val Acc: 0.4775
Saved model to out/MRD/ggnn_bi_fuse_dependency_ckpt/params.saved
Epoch: [6 / 500] | Time: 8.57s | Loss: 0.0193 |Train Acc: 0.4

Epoch    60: reducing learning rate of group 0 to 7.8125e-06.
Epoch: [60 / 500] | Time: 8.55s | Loss: 0.0044 |Train Acc: 0.8673 | Val Acc: 0.6712
EarlyStopping counter: 9 out of 10
Epoch: [61 / 500] | Time: 8.54s | Loss: 0.0042 |Train Acc: 0.8717 | Val Acc: 0.6685
EarlyStopping counter: 10 out of 10
Test examples: 990 | Time: 2.99s |  Test Acc: 0.6743
Total runtime: 1226.84s
